In [ ]:
# SISTEMA DE RECOMENDACION
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import SimpleImputer
import numpy as np

# Cargar los datos
data = pd.read_csv('df_modelo.csv')

# Seleccionar un subconjunto aleatorio de los datos
np.random.seed(42)
idx = np.random.choice(data.index, size=15000, replace=False)
data = data.loc[idx]

# Preprocesar los datos
mlb = MultiLabelBinarizer()
genres = pd.DataFrame(mlb.fit_transform(data['genres']), columns=mlb.classes_)
languages = pd.DataFrame(mlb.fit_transform(data['spoken_languages']), columns=mlb.classes_)
X = pd.concat([genres, languages, data['vote_average']], axis=1)

# Imputar los valores NaN en la matriz X
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Entrenar el modelo
model = NearestNeighbors(n_neighbors=6, algorithm='ball_tree')
model.fit(X)

def recomendacion(titulo: str):
    """
    Ingresar el nombre de pelicula y te recomienda las similares en una lista de 5 valores.\n
    Ejemplo  ===>  pelicula: Sabrina
    """
    # Encontrar el índice de la película
    idx = data[data['title'] == titulo].index[0]

    # Encontrar las películas más similares
    distances, indices = model.kneighbors(X[idx].reshape(1, -1))

    # Crear la lista de recomendaciones
    recomendaciones = []
    for i in range(1, len(distances.flatten())):
        recomendaciones.append(data.iloc[indices.flatten()[i]]['title'])

    return {'Lista recomendada': recomendaciones}

#recomendacion('Sabrina')